In [55]:
# MLP
#https://www.jmlr.org/papers/volume3/bengio03a/bengio03a.pdf

In [56]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [57]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [58]:
len(words)

32033

In [59]:
# build the vocabulary of characters and mappings to/from integers
chars = sorted(list(set(''.join(words))))
stoi = { s: i+1 for i, s in enumerate(chars) }
stoi['.'] = 0
itos = { i: s for s, i in stoi.items() }
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [60]:
# build the dataset

block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], [] # X are input to the NN, Y are the labels for the each example in X
#for w in words:
for w in words[:5]:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

... ---> e
..e ---> m
.em ---> m
emm ---> a
mma ---> .
... ---> o
..o ---> l
.ol ---> i
oli ---> v
liv ---> i
ivi ---> a
via ---> .
... ---> a
..a ---> v
.av ---> a
ava ---> .
... ---> i
..i ---> s
.is ---> a
isa ---> b
sab ---> e
abe ---> l
bel ---> l
ell ---> a
lla ---> .
... ---> s
..s ---> o
.so ---> p
sop ---> h
oph ---> i
phi ---> a
hia ---> .


In [61]:
# input is 32 by 3 since we have 32 examples of context length 3
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([32, 3]), torch.int64, torch.Size([32]), torch.int64)

In [62]:
# starting to implement the embedded lookup table

C = torch.randn((27,2)) # we have 27 characters in our vocabulary, and we map them to 2D vectors

In [63]:
emb = C[X]
emb.shape

torch.Size([32, 3, 2])

In [64]:
# implementing hidden layer
W1 = torch.randn((6, 100)) # 6 is the size of the embedding, 100 is the size of the hidden layer(kinda randomly chosen)
b1 = torch.randn((100)) # bias for the hidden layer

In [65]:
#emb @ W1 + b1   we cant exactly do this, since the dimensions dont match
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], dim=1).shape

torch.Size([32, 6])

In [66]:
# since the block size is 3, we can just concatenate the embeddings like this but what if the block size is 10?
# so unbind allows us to split the tensor into a list of tensors
torch.cat(torch.unbind(emb, dim=1), dim=1).shape

# so thats also a way to do it, but we can also 'view' the whole new tensor as a 2D tensor like whole new storage

torch.Size([32, 6])

In [67]:
emb.shape

torch.Size([32, 3, 2])

In [68]:
emb.view(32, 6) == torch.cat(torch.unbind(emb, dim=1), dim=1)

tensor([[True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, True, True],
        [True, True, True, True, T

In [69]:
# so creating the h like this allows us to do the matrix multiplication and addition of the bias
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # -1 means infer the size from the other dimensions so this is 32 by 100
# this would work even if its not 32, but some other number of examples

# this is more efficient than the previous method because it just manipulate the view of the tensor, not the actual data

In [72]:
h

tensor([[-0.7502,  0.9254, -0.3604,  ...,  0.7010, -0.9922,  0.3060],
        [ 0.3557,  0.9749,  0.4958,  ...,  0.4826, -0.9838,  0.2424],
        [-0.5637, -0.5824, -0.6801,  ..., -0.5704, -0.9993,  0.7410],
        ...,
        [-0.9908, -0.9978, -0.9587,  ..., -0.9735, -0.9993, -0.5138],
        [ 0.9984, -0.9739, -0.9998,  ..., -0.9990, -0.9984, -0.9838],
        [-0.9988, -0.9975, -0.9992,  ..., -0.9937, -0.9951, -0.9995]])

In [74]:
# creating the final layer
W2 = torch.randn((100, 27)) # 27 is the size of the vocabulary
b2 = torch.randn((27))

In [78]:
# output layer, logits are raw predictions which come out of the last layer of the nn
logits = h @ W2 + b2 # 32 by 27

In [80]:
logits.shape

torch.Size([32, 27])

In [82]:
counts = logits.exp() # we take the exponential of the logits to get the probabilities

In [84]:
probs = counts / counts.sum(dim=1, keepdim=True) # we normalize the counts to get the probabilities

In [88]:
probs.shape

torch.Size([32, 27])

In [90]:
# now via negative log likelihood loss we can calculate the loss
loss = -probs[torch.arange(32), Y].log().mean() # we take the negative log of the probabilities of the correct characters
loss # now thats the loss we want to minimize to get the nn to predict the next character correctly

tensor(16.8804)

In [92]:
# ---- Summary in respectable form ----
X.shape, Y.shape # dataset

(torch.Size([32, 3]), torch.Size([32]))

In [94]:
g = torch.Generator().manual_seed(42) # seed for reproducibility
C = torch.randn((27, 6), generator=g) # embedding layer

W1 = torch.randn((6, 100), generator=g) # hidden layer
b1 = torch.randn((100), generator=g) # bias for hidden layer

W2 = torch.randn((100, 27), generator=g) # output layer
b2 = torch.randn((27), generator=g) # bias for output layer

params = [C, W1, b1, W2, b2] # all the parameters of the model

In [96]:
sum(p.nelement() for p in params) # total number of parameters

3589

In [97]:
emb = C[X] # embedding lookup table
h = torch.tanh(emb.view(-1, 6) @ W1 + b1) # hidden layer
logits = h @ W2 + b2 # output layer

counts = logits.exp() # unnormalized probabilities
probs = counts / counts.sum(dim=1, keepdim=True) # normalized probabilities
loss = -probs[torch.arange(32), Y].log().mean() # negative log likelihood loss
loss

tensor(14.1915)